In [14]:
import re
import string 
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer 
from nltk.tokenize import TweetTokenizer 
import numpy as np 

In [15]:
# Preprocessing tweets
def process_tweet(tweet):

    # Removing old style retweet text 'RT'
    tweet2 = re.sub(r'^RT[/s]','',tweet)

    # Removing URL's 
    tweet2 = re.sub(r'https?:\/\/.*[\r\n]*','',tweet2)

    # Removing '#'
    tweet2 = re.sub(r'#' ,'',tweet2)

    # Instantiate tokenizer class
    tokenizer = TweetTokenizer(preserve_case=False ,strip_handles=True ,reduce_len=True)

    # Tokenize tweets
    tweet_tokens = tokenizer.tokenize(tweet2)

    # Import stopwords
    stopwords_english = stopwords.words("english")

    # List of words without stopwords 
    tweets_clean = []
    for word in tweet_tokens:
        if word not in stopwords_english and word not in string.punctuation:
            tweets_clean.append(word)

    # Instantiate stemming class 
    stemmer = PorterStemmer()

    # List of stems in tweets 
    tweet_stems = []
    for word in tweets_clean:
        stem_word = stemmer.stem(word)
        tweet_stems.append(stem_word)

    return tweet_stems

In [16]:
# Frequency generating function
def build_freqs(tweets, ys):
    yslist = np.squeeze(ys).tolist() # squeezes ys into a 1-D array and then converts it into list.

    freqs = {}
    for y,tweet in zip(yslist ,tweets):  # zip creates pairs of elements.
        for word in process_tweet(tweet):
            pair = (word, y)
            freqs[pair] = freqs.get(pair, 0) + 1  # if pair is not present in the freqs dictionary 
                                                  # then initialize it to 0 and then update frequency
                                                  # count by 1. 
    return freqs


In [18]:
def extract_features(tweet ,freqs):

    # pre-process tweet
    word_l = process_tweet(tweet)

    # 3 elements in the form of a 1x3 matrix
    x = np.zeros((1,3))

    # bias term is set to 1
    x[0,0] = 1

    # looping through each word
    for word in word_l:

        # increment the word count for positive label 1
        x[0,1] += freqs.get((word,1),0)   # returns 0 as default value if positive label is not present.

        # increment the word count for negative label 0
        x[0,2] += freqs.get((word,0),0)
    
    assert(x.shape == (1,3)) # Raises error if condition is not true

    return x

In [19]:
import nltk

nltk.download('twitter_samples')

from nltk.corpus import twitter_samples
all_positive_tweets=twitter_samples.strings('positive_tweets.json')
all_negative_tweets=twitter_samples.strings('negative_tweets.json')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\ayush\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


In [20]:
# Splitting the data into testing data and training data

test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg 
test_x = test_pos + test_neg 

# Combining the data for positive and negative labels
train_y = np.append(np.ones((len(train_pos) ,1)) , np.zeros((len(train_neg) ,1)) ,axis=0)
test_y = np.append(np.ones((len(test_pos) ,1)) , np.zeros((len(test_neg) ,1)) ,axis=0)

In [21]:
# Build frequency dictionary from training data
freqs = build_freqs(train_x, train_y)

In [22]:
# Build training feature matrix
X_train = np.zeros((len(train_x), 3))
for i, tweet in enumerate(train_x):
    X_train[i, :] = extract_features(tweet, freqs)

# Build test feature matrix
X_test = np.zeros((len(test_x), 3))
for i, tweet in enumerate(test_x):
    X_test[i, :] = extract_features(tweet, freqs)


In [23]:
def train_naive_bayes(freqs, train_x, train_y):
    # Calculate priors
    N_pos = np.sum(train_y)
    N_neg = len(train_y) - N_pos
    logprior = np.log(N_pos / N_neg)
    
    # Build vocabulary
    vocab = set([pair[0] for pair in freqs.keys()])
    V = len(vocab)
    
    # Count total number of words in positive and negative tweets
    word_counts_pos = 0
    word_counts_neg = 0
    for pair in freqs:
        if pair[1] == 1:
            word_counts_pos += freqs[pair]
        else:
            word_counts_neg += freqs[pair]
    
    # Likelihoods with Laplace smoothing
    loglikelihood = {}
    for word in vocab:
        freq_pos = freqs.get((word, 1), 0)
        freq_neg = freqs.get((word, 0), 0)
        # Laplace smoothing
        p_w_pos = (freq_pos + 1) / (word_counts_pos + V)
        p_w_neg = (freq_neg + 1) / (word_counts_neg + V)
        loglikelihood[word] = np.log(p_w_pos / p_w_neg)
    
    return logprior, loglikelihood


In [24]:
def naive_bayes_predict(tweet, logprior, loglikelihood):
    words = process_tweet(tweet)
    p = logprior
    for word in words:
        if word in loglikelihood:
            p += loglikelihood[word]
    return 1 if p > 0 else 0

In [25]:
logprior, loglikelihood = train_naive_bayes(freqs, train_x, train_y)

In [26]:
# Predict on test set
y_pred = []
for tweet in test_x:
    y_pred.append(naive_bayes_predict(tweet, logprior, loglikelihood))
y_pred = np.array(y_pred)

# Calculate accuracy
accuracy = np.mean(y_pred == test_y.flatten()) * 100
print(f"Test Accuracy: {accuracy:.2f}%")


Test Accuracy: 99.40%
